# [실습] HuggingFace 공개 LLM을 활용한 Agentic Work 만들기

HuggingFace에 게시된 공개 모델들의 경우, `bind_tools`와 같은 기능들이 제대로 연동되지 않거나,   
자체적으로 Tool Calling 기능이 없는 경우가 많습니다.   

이를 해결하기 위해, Tool Calling 방법을 직접 구성하고 Agentic Work를 구현해 보겠습니다.


## 라이브러리 설치


이번 실습은 무료 코랩(T4 GPU)이 아닌 고성능 GPU 라이브러리에서 진행합니다.   
무료 코랩으로 진행하시는 경우, GPU 성능의 한계로 실행이 오래 걸릴 수 있습니다.   
(아래 코드에서, **T4 사용시 해제** 부분을 참고하세요!)

<br><br>


In [ ]:
!pip install transformers bitsandbytes accelerate -q

In [ ]:
!pip install --upgrade langchain_experimental langgraph langchain langchain_community langchain_huggingface dotenv -q

In [ ]:
# Flash Attention: 리눅스 전용 설치방법
# 무료 코랩(T4)에서는 설치 X

# Windows 설치는 https://github.com/kingbri1/flash-attention/releases 참고
!pip install flash-attn --no-build-isolation -q

설치할 라이브러리가 많으므로, 가급적 설치 후 세션 재시작을 수행해 주세요.

허깅페이스 토큰을 인증합니다.    

https://huggingface.co/settings/tokens 에서 Read 권한 토큰을 발급받습니다.   
아래 코드에서는, env 파일에 `HF_READ_TOKEN`으로 저장했습니다.

In [ ]:
from dotenv import load_dotenv

In [ ]:
load_dotenv('env') #.env 파일인 경우 .env로 변경

In [ ]:
import os
os.environ['HF_TOKEN']

In [ ]:
import os

from huggingface_hub import login

# 허깅페이스 토큰 로그인
login(token=os.environ['HF_TOKEN'])

### 1. Gemma 3

불러올 모델은 구글의 Gemma-3-12b-it 입니다.   
실제 크기는 24GB 정도이지만, 4Bit 양자화를 통해 8~9GB 크기로 로드합니다.

In [ ]:
model_id = "google/gemma-3-12b-it"
print(f"# Model ID: {model_id}")

트랜스포머 라이브러리를 이용해 모델을 불러옵니다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch

# 양자화 옵션: 4비트
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    # nf4 양자화

    bnb_4bit_compute_dtype="bfloat16", # 계산시에는 기존 자료형 bf16 사용
    bnb_4bit_use_double_quant=True # 이중 양자화
)


# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,

    torch_dtype=torch.bfloat16, # Gemma 3은 bf16 모델입니다!
    device_map={"":0},  # 0번 GPU에 할당

    quantization_config = quantization_config,
    # 양자화 불필요시 제거

    attn_implementation = 'eager'
    # flash attention 설정
    # Gemma Series가 아닌 경우 'eager'를 "flash_attention_2" 로 변경
    # 미지원/사용하지 않는 경우 제거 (T4 등의 구버전 GPU 등)

)

# if hasattr(model, 'language_model'):
#     model = model.language_model
# Multimodal 모델의 경우, Language Model만 선택할 수도 있으나
# 비전 모델의 사이즈가 매우 작기 때문에 큰 차이는 없음

# Train X (eval)
model.eval()


모델을 불러온 뒤에는 Text-Generation 파이프라인을 구성합니다.   
이 때, 적절한 샘플링 파라미터를 설정하기 위해 모델 홈페이지를 참고할 수 있습니다.

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

# Sampling 파라미터 설정: https://huggingface.co/google/gemma-3-12b-it
# 홈페이지 권장 사양이지만, 적절하게 바꿔도 됨
gen_config = dict(
    do_sample=True,
    max_new_tokens=2048,
    repetition_penalty = 1.05,

    temperature = 1.0,
    top_p = 0.95,
    top_k = 64
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    # return_full_text=True <-- 입력 프롬프트를 포함한 전체 출력하기
    **gen_config
    )

모델의 토크나이저를 확인하여, 채팅 템플릿 구성을 확인합니다.   
전체를 이해할 필요는 없지만, Tool이나 Function이 있는지 확인합니다.

In [ ]:
print(tokenizer.chat_template)
# Tool 미지원 😣

모델을 랭체인과 연동합니다.   

In [ ]:
base_llm = HuggingFacePipeline(pipeline=pipe, pipeline_kwargs=gen_config)
# base_llm: 입력 전 Tokenizer로 템플릿 변환 필수, 스트리밍 가능

llm = ChatHuggingFace(llm=base_llm, tokenizer=tokenizer)
# llm: 자동 템플릿 변환(ChatGoogleGenAI와 동일), 스트리밍 불가

In [ ]:
def convert_chat(messages, add_generation_prompt = True):
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=add_generation_prompt)

example = [{'role':'user', 'content':'안녕'}]
convert_chat(example)

In [ ]:
for s in base_llm.stream(convert_chat(example)):
    print(s, end='')

In [ ]:
llm.invoke("안녕?")

## 2. HuggingFace LLM과 툴 연동하기

먼저 툴을 설정합니다.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.tools.python.tool import PythonREPLTool
from datetime import datetime

tavily_search = TavilySearchResults(
    max_results=3)

repl_tool = PythonREPLTool()
repl_tool.invoke("for i in range(10): print(i)")


@tool
def current_date() -> str:
    "현재 날짜를 %y-%m-%d 형식으로 반환합니다."
    return datetime.now().strftime("%Y-%m-%d")



tools = [tavily_search, repl_tool, current_date]


`ChatHuggingFace()`에도 `bind_tools()`이 구현되어 있으나,   
실제로 실행되지 않는 경우가 많습니다.

In [ ]:
llm_with_tools = llm.bind_tools(tools)
llm_with_tools.invoke("오늘 날짜가 어떻게 되니?")

In [ ]:
llm_with_tools.invoke("너는 어떤 툴이나 함수를 가지고 있니?")

이런 경우, 툴을 실행하기 위해서는 직접 시스템 메시지를 구성해야 합니다.   
Tool 역할 또한 존재하지 않으므로 User 역할을 툴 대용으로 사용합니다.

In [ ]:
llm_with_tools.kwargs['tools']

툴 설명이 담긴 문자열을 구성합니다.

In [ ]:
tool_desc = str('\n---\n'.join([str(x) for x in llm_with_tools.kwargs['tools']]))
print(tool_desc)

시스템 프롬프트를 구성합니다.   
성능에 매우 중요한 영향을 미치므로, 영어로 작성했습니다.

In [ ]:
system_prompt = f'''
You are a helpful assistant with tools below.
You can decide whether to invoke any functions or not.
If you decide to use any of tools.
print name and required parameters of the tool within a json blob correctly.
For python code, Return the object as a raw dictionary, without escaping quotes or newlines.

for tool use: wrap your output within ```tool_code```.

Example:
```tool_code
{{"name":'name of tool', "arguments":{{List of apparent argument and parameters}}}}
```

When the output of the tool is provided, it will be wrapped within ``tool_output```
Answer accordingly from the result of the tool output.

The question might need some sequential, multiple tool execution.
Think Step by Step.

The following tools are available:
{tool_desc}'''

시스템 프롬프트에 들어가야 하는 내용은 다음과 같습니다.
- Tool Format
- Tool Call Format
- Tool Result Format

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage


messages = [SystemMessage(system_prompt),
            HumanMessage('오늘 날짜가 며칠이니?')]


In [ ]:

response = llm.invoke(messages)
messages.append(response)
response


In [ ]:
print(response.content)

tool_code를 받았으니, 해당 내용을 파싱합니다.

In [ ]:
import ast


def parse_tool(text):
    try:
        text = text.split('```tool_code\n')[1].split('\n```')[0]
        # tool_code로 wrap된 중간 코드 추출

        parsed = ast.literal_eval(text)
        # Dict 형태의 값 변환 (json load와 유사)

        name = parsed.get('name')
        arguments = parsed.get('arguments', {})
        # name과 argument return
        return {'name':name, 'arguments':arguments}
    except (ValueError, SyntaxError):
        return None

result = parse_tool(response.content)
name,arguments = result['name'], result['arguments']
name, arguments

툴 실행을 연결합니다.

In [ ]:
# 툴 이름과 툴 연결
tool_dict = {tool.name: tool for tool in tools}

def execute_tool(name, arguments):
    # 툴 실행한 뒤 tool_output으로 wrap
    result = f'''```tool_output
{tool_dict[name].invoke(arguments)}
```'''
    return result

tool_result = execute_tool(**parse_tool(response.content))

print(tool_result)

In [ ]:
messages.append(HumanMessage(tool_result))
messages[1:]
# 질문 + Tool 요청 + Tool 결과

In [ ]:
# 결과 해석
response = llm.invoke(messages)
response

In [ ]:
messages = [SystemMessage(system_prompt),
            HumanMessage('2025년 4월 발표된 GPT-4.1 모델이 어떤 모델이야? 한국어로 설명해줘.')]
response = llm.invoke(messages)
messages.append(response)
response

In [ ]:
print(response.content)

In [ ]:
tool_result = execute_tool(**parse_tool(response.content))
tool_result

In [ ]:
messages[-1]

In [ ]:
messages.append(HumanMessage(tool_result))
response = llm.invoke(messages)
response

일반적인 입출력 관계의 툴은 이와 같은 방식으로 간단하게 구성할 수 있습니다.   
(만약, Python_REPL과 같이 argument가 복잡한 툴을 수행하는 경우에는   
별도의 함수로 변환하거나 결과물을 수정하는 작업이 필요할 수 있습니다.)

해당 구현을 통해, ReAct Agent 구조를 만들어 보겠습니다.    
bind_tools가 없기 때문에, 기존의 Tool Message를 사용하기 어렵습니다.

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages : Annotated[list, add_messages]   # 메시지 맥락을 저장하는 리스트


In [ ]:
from langchain_core.messages import ToolMessage

tool_list = {tool.name: tool for tool in tools}
# tool 목록 dict로 생성

def tool_node(state):
    tool_outputs = []
    tool_call_msgs = state['messages'][-1]
    # 마지막 메시지: 툴 콜링 메시지
    if '```tool_code' in tool_call_msgs.content:
        tool_result = execute_tool(**parse_tool(tool_call_msgs.content))
        # tool 실행 결과 얻기 (결과는 ```tool_output```)
        tool_outputs.append(HumanMessage(tool_result))

    return {'messages': tool_outputs}

def agent(state):
    system_prompt = SystemMessage(f'''
You are a helpful assistant with tools below.
You can decide whether to invoke any functions or not.
If you decide to use any of tools.
print name and required parameters of the tool within a json blob correctly.
For python code, Return the object as a raw dictionary, without escaping quotes or newlines.

for tool use: wrap your output within ```tool_code```.

Example:
```tool_code
{{"name":'name of tool', "arguments":{{List of apparent argument and parameters}}}}
```

When the output of the tool is provided, it will be wrapped within ``tool_output```
Answer accordingly from the result of the tool output.

The question might need some sequential, multiple tool execution.
Think Step by Step.

The following tools are available:
{tool_desc}


Answer in Korean.''')


    response = llm.invoke([system_prompt] + state["messages"])
    return {'messages': response}

def tool_needed(state):

    last_msg = state['messages'][-1]
    if '```tool_code' in last_msg.content: # 툴 콜링이 필요하면
        return "continue"
    else:
        return "finish"

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)

builder.add_node("agent", agent)
builder.add_node("tools", tool_node)

builder.add_edge(START, 'agent'),
builder.add_conditional_edges("agent",
                              tool_needed,
                               {"continue": "tools","finish": END})
builder.add_edge("tools", "agent")

In [ ]:
graph = builder.compile()
graph # 생성한 그래프 시각화

In [ ]:
response = graph.invoke({'messages':[HumanMessage(content="오늘 날짜에 태어난 유명인들 조사해서 알려줘.")]})
response

Gemma 3는 Native Tool Call을 지원하는 모델은 아닙니다.  
하지만, 어느 정도 함수 실행 능력을 보이는데요.   

마찬가지로, Gemma 3 이외의 LLM 모델을 사용하는 경우에도,  
각각의 Tool 스펙에 대한 문서를 참고하여 진행하실 수 있으나, 성능은 조금 떨어질 수 있습니다.   
Ex) Phi-4 https://huggingface.co/microsoft/Phi-4-mini-instruct

<br><br>
이번에는 Qwen 2.5 7B를 이용해 진행해 보겠습니다.

**세션 초기화 후 여기서부터 다시 시작해 주세요!**


In [ ]:
from dotenv import load_dotenv
load_dotenv('env')

In [ ]:
model_id = "Qwen/Qwen2.5-7B-Instruct"
print(f"# Model ID: {model_id}")
# Full Precision 로드: 15.6GB
# GPU 부족시 양자화

In [ ]:
from transformers import BitsAndBytesConfig

# 양자화 옵션: 4비트
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    # nf4 양자화

    bnb_4bit_compute_dtype="bfloat16", # 계산시에는 기존 자료형 bf16 사용
    bnb_4bit_use_double_quant=True # 이중 양자화
)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,

    torch_dtype='auto',
    device_map={"":0},  # 0번 GPU에 할당

    quantization_config  = quantization_config,

    # attn_implementation = 'flash_attention_2'
    # flash attention 설정
    # 미지원/사용하지 않는 경우 제거 (T4 등의 구버전 GPU 등)

)

# Train X (eval)
model.eval()


In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

# Sampling 파라미터 설정: https://huggingface.co/Qwen/Qwen2.5-7B-Instruct
# 홈페이지 권장 사양이지만, 적절하게 바꿔도 됨
gen_config = dict(
    do_sample=True,
    max_new_tokens=2048,
    repetition_penalty = 1.05,

    temperature = 0.1,
    top_p = 0.8,
    top_k = 20
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    # return_full_text=True <-- 입력 프롬프트를 포함한 전체 출력하기
    **gen_config
    )

Qwen 2.5 는 Tool 기능이 포함되어 있습니다.

In [ ]:
print(tokenizer.chat_template)

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.tools.python.tool import PythonREPLTool
from datetime import datetime

tavily_search = TavilySearchResults(
    max_results=3)

repl_tool = PythonREPLTool()


@tool
def current_date() -> str:
    "현재 날짜를 %y-%m-%d 형식으로 반환합니다."
    return datetime.now().strftime("%Y-%m-%d")



tools = [tavily_search, repl_tool, current_date]


In [ ]:
base_llm = HuggingFacePipeline(pipeline=pipe, pipeline_kwargs=gen_config)
# base_llm: 입력 전 Tokenizer로 템플릿 변환 필수, 스트리밍 가능

llm = ChatHuggingFace(llm=base_llm, tokenizer=tokenizer)
# llm: 자동 템플릿 변환(ChatGoogleGenAI와 동일), 스트리밍 불가

툴 기능이 있어도, bind_tools는 사용이 어렵습니다.

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
llm_with_tools.invoke("오늘 날짜가 어떻게 되니? 툴을 사용해서 알려줘.")

툴을 json list로 전달하여 템플릿에 적용합니다.

In [ ]:
import json
tool_desc = llm_with_tools.kwargs['tools']
print(tool_desc)

토크나이저에 tool을 전달하여, 전반적인 템플릿을 확인합니다.

In [ ]:
def convert_chat_with_tools(messages, tools=None, add_generation_prompt = True):
    return tokenizer.apply_chat_template(messages, tools=tools, tokenize=False, add_generation_prompt=add_generation_prompt)

example = [{'role':'user', 'content':'오늘 날짜가 어떻게 돼?'}]
print(convert_chat_with_tools(example, tools=tool_desc))

In [ ]:
for s in base_llm.stream(convert_chat_with_tools(example, tools=tool_desc)):
    print(s, end='')

해당 내용을 가져가서, 시스템 프롬프트를 구성합니다.

In [ ]:
# 최상의 결과를 위해, 토크나이저의 템플릿을 최대한 따릅니다.
system_prompt = f'''
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{tool_desc}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{{"name": <function-name>, "arguments": <args-json-object>}}
</tool_call>
'''

Gemma와 동일하게 실행해 보겠습니다.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

messages = [SystemMessage(system_prompt),
            HumanMessage('오늘 날짜가 며칠이니?')]

response = llm.invoke(messages)
messages.append(response)
response

tool_call의 형식에 맞춰 코드를 수정합니다.

In [ ]:
import ast

def parse_tool(text):
    try:
        text = text.split('<tool_call>\n')[1].split('\n</tool_call>')[0]
        # tool_code로 wrap된 중간 코드 추출

        parsed = ast.literal_eval(text)
        # Dict 형태의 값 변환 (json load와 유사)
        name = parsed.get('name')
        arguments = parsed.get('arguments', {})
        # name과 argument return
        return {'name':name, 'arguments':arguments}
    except (ValueError, SyntaxError):
        return None

result = parse_tool(response.content)
name,arguments = result['name'], result['arguments']
name, arguments

툴을 실행하고, 그 결과는 ToolMessage로 전달합니다.

In [ ]:
# 툴 이름과 툴 연결
tool_dict = {tool.name: tool for tool in tools}

def execute_tool(name, arguments):
    # 툴 실행한 뒤 tool_output으로 wrap
    result = tool_dict[name].invoke(arguments)
    return str(result)

tool_result = execute_tool(**parse_tool(response.content))

tool_result

In [ ]:
messages.append(HumanMessage(tool_result))
messages[1:]
# 질문 + Tool 요청 + Tool 결과

In [ ]:
response = llm.invoke(messages)
messages.append(response)
response

동일한 구조로, 프롬프트를 수정하여 전체 에이전트를 구성할 수 있습니다.

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages : Annotated[list, add_messages]   # 메시지 맥락을 저장하는 리스트


In [ ]:
from langchain_core.messages import ToolMessage

tool_list = {tool.name: tool for tool in tools}
# tool 목록 dict로 생성

def tool_node(state):
    tool_outputs = []
    tool_call_msgs = state['messages'][-1]
    # 마지막 메시지: 툴 콜링 메시지
    if '<tool_call>' in tool_call_msgs.content:
        tool_result = execute_tool(**parse_tool(tool_call_msgs.content))
        # tool 실행 결과 얻기 (결과는 ```tool_output```)
        tool_outputs.append(HumanMessage(tool_result))

    return {'messages': tool_outputs}

def agent(state):
    system_prompt = SystemMessage(f'''
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.

# Tools

You may call a function to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{tool_desc}
</tools>

For function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{{"name": <function-name>, "arguments": <args-json-object>}}
</tool_call>

# Tool Usage Rules - VERY IMPORTANT!

1.  **Analyze the Task:** Carefully read the user's request and determine if any tools are needed.
2.  **One Tool At A Time:** If you need to use a tool, you **MUST** choose and call **only ONE** tool in your response. Do **NOT** issue multiple `<tool_call>` tags in a single turn.
3.  **Sequential Execution:** If the user's request requires information from multiple tool calls (e.g., searching for information and then getting the weather based on the search result), you **MUST** perform these calls sequentially.
    * First, call the **single** tool needed for the first step.
    * Wait for the result of that tool.
    * Then, based on the result, decide if another **single** tool call is necessary for the next step. Issue that call in a *new* response turn.

Answer in Korean.''')


    response = llm.invoke([system_prompt] + state["messages"])
    return {'messages': response}

def tool_needed(state):

    last_msg = state['messages'][-1]
    if '<tool_call>' in last_msg.content: # 툴 콜링이 필요하면
        return "continue"
    else:
        return "finish"

In [ ]:
# 메모리 세팅
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)

builder.add_node("agent", agent)
builder.add_node("tools", tool_node)

builder.add_edge(START, 'agent'),
builder.add_conditional_edges("agent",
                              tool_needed,
                               {"continue": "tools","finish": END})
builder.add_edge("tools", "agent")

In [ ]:
memory = MemorySaver()
graph = builder.compile(checkpointer = memory)

In [ ]:
config = {"configurable": {"thread_id": "1"}}


response = graph.invoke({'messages':[HumanMessage(content="오늘이 며칠이야?")]}, config)
response

In [ ]:
# sLLM: 결과가 Inconsistent...
config = {"configurable": {"thread_id": "2"}}

response = graph.invoke({'messages':[HumanMessage(content="오늘과 같은 날짜에 태어난 유명인들 조사해서 알려줘.")]}, config)
response

In [ ]:
# 검색 2
config = {"configurable": {"thread_id": "3"}}

response = graph.invoke({'messages':[HumanMessage(content="2025년 4월에 출시된 GPT-4.1 모델에 대해 소개해줘.")]},
                       config)
response

Tool을 사용하지 않는 Agentic Work의 경우에는 기존 LLM과 동일하게 실행이 가능합니다.

단, Transformers 라이브러리의 경우 동시 실행 등의 메커니즘이 최적화되어 있지 않기 때문에,   
이전의 Send()와 같이 병렬 실행이 필요한 문제에서는 결과가 제대로 나오지 않을 수 있습니다.    

이를 해결하는 방법은, Ollama나 vLLM과 같은 서빙 라이브러리를 사용하는 것입니다.